In [ ]:
import json
import outlines
import torch
from typing import List, Optional
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from datasets import load_dataset
from rich import print
import os
os.environ["TRANSFORMERS_VERBOSITY"] = "info"
load_dotenv()

In [ ]:
MODEL_NAME = os.getenv("MODEL_NAME")
MODEL_PATH = os.getenv("MODEL_PATH")
DATASET_NAME = os.getenv("DATASET_NAME")
SEED = int(os.getenv("SEED"))

In [ ]:
print(f"Loading dataset: {DATASET_NAME}")
print(f"Using seed: {SEED}")

In [ ]:
try:
    dataset = load_dataset(DATASET_NAME)
    print("Dataset loaded successfully!")
    print(dataset) # Print the dataset structure to see available splits
except Exception as e:
    print(f"Error loading dataset: {e}")
    print("Please ensure the dataset name is correct and you have an active internet connection.")


In [ ]:
full_dataset = dataset['train']
eval_dataset = full_dataset[-50:]

In [ ]:
class ResponseFormat(BaseModel):
    answer : str

In [ ]:
if torch.cuda.is_available():
    print("GPU is Used!")
else:
    print("CPU is Used!")

In [ ]:
model = outlines.models.transformers(MODEL_NAME,
                                     device="cuda",
                                     model_kwargs={"temperature":0.7,
                                                    "top_k":40,
                                                    "top_p":0.95},)

In [ ]:
inputs = eval_dataset['question'][:5]

In [ ]:
schema_as_str = json.dumps(ResponseFormat.model_json_schema())
generator = outlines.generate.json(model, schema_as_str)

In [ ]:
output = generator(inputs)
print(json.dumps(output,indent=4))

In [ ]:
for i, item in enumerate(output):
    print(f"Input: {inputs[i]}")
    print(f"Output: {item['answer']}")
    print("-" * 40)

In [ ]:
output[0]['answer']

In [ ]:
print(eval_dataset['answer'][:5])